In [1]:
import sqlite3

def generate_user_table():
    """
    Establishes a connection to the RegistrationDB, creates a user table,
    and inserts initial user data.
    """
    try:
        # Establish a connection to the database (creates the database if it doesn't exist)
        conn = sqlite3.connect('RegistrationDB.db')
        cursor = conn.cursor()

        # Create the user table if it doesn't exist
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS user (
                email varchar(100) PRIMARY KEY,
                name varchar(50),
                password varchar(30)
            )
        ''')

        # Insert initial user data
        users = [
            ('ywbaek@perscholas.org', 'young', 'letsgomets'),
            ('mcordon@perscholas.org', 'marcial', 'perscholas'),
            ('mhaseeb@perscholas.org', 'haseeb', 'platform'),
        ]
        cursor.executemany('INSERT OR IGNORE INTO user VALUES (?, ?, ?)', users) # Use OR IGNORE to avoid errors if the users are already there.

        # Commit the changes and close the connection
        conn.commit()
        print("User table generated and initial data inserted successfully.")

    except sqlite3.Error as e:
        print(f"Error: {e}")
        if conn:
            conn.rollback()  # Rollback changes in case of an error
    finally:
        if conn:
            conn.close()

def get_all_users():
    """
    Establishes a connection to the RegistrationDB and prints out the
    email, name, and password of all users.
    """
    try:
        # Establish a connection to the database
        conn = sqlite3.connect('RegistrationDB.db')
        cursor = conn.cursor()

        # Retrieve all users from the user table
        cursor.execute('SELECT email, name, password FROM user')
        users = cursor.fetchall()

        # Print out the user data
        if not users:
            print("No users found in the database.")
        else:
            print("All Users:")
            for user in users:
                print(f"Email: {user[0]}, Name: {user[1]}, Password: {user[2]}")

    except sqlite3.Error as e:
        print(f"Error: {e}")
    finally:
        if conn:
            conn.close()

def get_user_by_name(name):
    """
    Establishes a connection to the RegistrationDB and prints out the
    email and password of the user with the given name.
    """
    try:
        # Establish a connection to the database
        conn = sqlite3.connect('RegistrationDB.db')
        cursor = conn.cursor()

        # Retrieve the user with the given name
        cursor.execute('SELECT email, password FROM user WHERE name = ?', (name,))
        user = cursor.fetchone()

        # Print out the user data
        if user:
            print(f"User found with name '{name}':")
            print(f"Email: {user[0]}, Password: {user[1]}")
        else:
            print(f"User with name '{name}' not found in the database.")

    except sqlite3.Error as e:
        print(f"Error: {e}")
    finally:
        if conn:
            conn.close()

def validate_user(email, password):
    """
    Establishes a connection to the RegistrationDB and returns True if a user
    exists with the given email and password. Returns False otherwise.
    """
    try:
        # Establish a connection to the database
        conn = sqlite3.connect('RegistrationDB.db')
        cursor = conn.cursor()

        # Check if a user exists with the given email and password
        cursor.execute('SELECT 1 FROM user WHERE email = ? AND password = ?', (email, password,))
        user_exists = cursor.fetchone()

        # Return True if a user exists, False otherwise
        return user_exists is not None

    except sqlite3.Error as e:
        print(f"Error: {e}")
        return False  # Return False in case of an error
    finally:
        if conn:
            conn.close()

def update_user(email, name, password):
    """
    Establishes a connection to the RegistrationDB and updates the name and
    password of the user with the given email. Returns True if successful,
    False otherwise.
    """
    try:
        # Establish a connection to the database
        conn = sqlite3.connect('RegistrationDB.db')
        cursor = conn.cursor()

        # Update the user with the given email
        cursor.execute('UPDATE user SET name = ?, password = ? WHERE email = ?', (name, password, email))
        rows_affected = cursor.rowcount # Get the number of rows affected by the update

        # Commit the changes and close the connection
        conn.commit()

        # Return True if successful, False otherwise
        return rows_affected > 0

    except sqlite3.Error as e:
        print(f"Error: {e}")
        if conn:
            conn.rollback()  # Rollback changes in case of an error
        return False  # Return False in case of an error
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    # Test the functions
    generate_user_table()  # Create the table and insert initial data
    get_all_users()
    get_user_by_name('marcial')
    print(f"User validation: {'ywbaek@perscholas.org' and 'letsgomets'}: {validate_user('ywbaek@perscholas.org', 'letsgomets')}")
    print(f"User validation: non_existent@example.com and 'password': {validate_user('non_existent@example.com', 'password')}")
    updated = update_user('ywbaek@perscholas.org', 'Young Baek', 'newpassword')
    if updated:
        print("User updated successfully.")
    else:
        print("User update failed.")
    get_all_users() # show the updated user.


User table generated and initial data inserted successfully.
All Users:
Email: ywbaek@perscholas.org, Name: young, Password: letsgomets
Email: mcordon@perscholas.org, Name: marcial, Password: perscholas
Email: mhaseeb@perscholas.org, Name: haseeb, Password: platform
User found with name 'marcial':
Email: mcordon@perscholas.org, Password: perscholas
User validation: letsgomets: True
User validation: non_existent@example.com and 'password': False
User updated successfully.
All Users:
Email: ywbaek@perscholas.org, Name: Young Baek, Password: newpassword
Email: mcordon@perscholas.org, Name: marcial, Password: perscholas
Email: mhaseeb@perscholas.org, Name: haseeb, Password: platform


In [6]:
import requests

def bodyTemperature(doctorName, diagnosisId):

    base_url = "https://jsonmock.hackerrank.com/api/medical_records"
    temperatures = []                                              

    try:
        # first page 
        response = requests.get(base_url)  #
        response.raise_for_status()    # HANDLE HTTP ERRORS
        result = response.json()          # PARSE THE JSON
        total_pages = result.get("total_pages", 0)

        # Loop the other page
        for page in range(1, total_pages + 1):
            url = f"{base_url}?page={page}"
            page_response = requests.get(url)
            page_response.raise_for_status()
            data = page_response.json().get("data", [])

            for record in data:
                try:
                    if (record.get("doctor", {}).get("name") == doctorName and 
                        record.get("diagnosis", {}).get("id") == diagnosisId):

                        # Get temperature if valid
                        temperature = record.get("vitals", {}).get("bodyTemperature")
                        if isinstance(temperature, (int, float)):
                            temperatures.append(temperature)

                except (KeyError, TypeError):
                    continue  # Skip malformed records

        if not temperatures:
            return [None, None]

        # min and max body temperatures
        return [int(min(temperatures)), int(max(temperatures))]

    except (requests.RequestException, ValueError) as e:
        print("Error fetching or processing data:", e)
        return [None, None]


# cases

if __name__ == "__main__":
    doctor_name = "Dr Arnold Bullock"
    diagnosis_id = 3
    result = bodyTemperature(doctor_name, diagnosis_id)
    print(f"Doctor: {doctor_name}")
    print("Minimum Body Temperature:", result[0])
    print("Maximum Body Temperature:", result[1])

    doctor_name = "Dr Allysa Ellis"
    diagnosis_id = 4
    result = bodyTemperature(doctor_name, diagnosis_id)
    print(f"Doctor: {doctor_name}")
    print("Minimum Body Temperature:", result[0])
    print("Maximum Body Temperature:", result[1])

    doctor_name = "Dr Adam"
    diagnosis_id = 5
    result = bodyTemperature(doctor_name, diagnosis_id)
    print(f"Doctor: {doctor_name}")
    print("Minimum Body Temperature:", result[0])
    print("Maximum Body Temperature:", result[1])


Doctor: Dr Arnold Bullock
Minimum Body Temperature: 99
Maximum Body Temperature: 103
Doctor: Dr Allysa Ellis
Minimum Body Temperature: 97
Maximum Body Temperature: 99
Doctor: Dr Adam
Minimum Body Temperature: None
Maximum Body Temperature: None
